# 关系抽取

一旦文本中的命名实体被识别，我们就可以提取它们之间存在的关系，通常我们会寻找指定类型的命名实体之间的关系。进行这一任务的方法之一时首先寻找所有（X，a，Y）形式的三元组，其中 X 和 Y 是指定类型的命名实体，a 表示 X 和 Y 之间关系的字符串，然后我们可以使用正则表达式从 a 中抽出我们正在查找的关系。

下面的例子搜索包含词 in 的字符串，特殊的正则表达式 (?!\b.+ing) 是一个否则预测先行断言，允许我们忽略 success in supervising the transition of 这样 in 后面跟一个动名词的字符串：

In [1]:
import nltk
import re

IN = re.compile(r'.*\bin\b(?!\b.+ing)')
for doc in nltk.corpus.ieer.parsed_docs('NYT_19980315'):
    for rel in nltk.sem.extract_rels('ORG', 'LOC', doc, corpus='ieer', pattern=IN):
        print(nltk.sem.rtuple(rel))

[ORG: 'WHYY'] 'in' [LOC: 'Philadelphia']
[ORG: 'McGlashan &AMP; Sarrail'] 'firm in' [LOC: 'San Mateo']
[ORG: 'Freedom Forum'] 'in' [LOC: 'Arlington']
[ORG: 'Brookings Institution'] ', the research group in' [LOC: 'Washington']
[ORG: 'Idealab'] ', a self-described business incubator based in' [LOC: 'Los Angeles']
[ORG: 'Open Text'] ', based in' [LOC: 'Waterloo']
[ORG: 'WGBH'] 'in' [LOC: 'Boston']
[ORG: 'Bastille Opera'] 'in' [LOC: 'Paris']
[ORG: 'Omnicom'] 'in' [LOC: 'New York']
[ORG: 'DDB Needham'] 'in' [LOC: 'New York']
[ORG: 'Kaplan Thaler Group'] 'in' [LOC: 'New York']
[ORG: 'BBDO South'] 'in' [LOC: 'Atlanta']
[ORG: 'Georgia-Pacific'] 'in' [LOC: 'Atlanta']


搜索关键字 in 执行的相当不错，虽然它的检索结果有时候也会误报。

CoNLL 2002 命名实体语料库的荷兰语部分不止包含命名实体标注，也包含词性标注，这允许我们设计对这些标记敏感的模式，如下面的例子所示，通过 [extract_rels](https://www.nltk.org/_modules/nltk/sem/relextract.html#extract_rels) 方法过滤出满足条件的命名实体，并用 [clause](https://www.nltk.org/_modules/nltk/sem/relextract.html#clause) 方法展示出来：

In [2]:
from nltk.corpus import conll2002

vnv = """
    (
    is/V|    # 3rd sing present and
    was/V|   # past forms of the verb zijn ('be')
    werd/V|  # and also present
    wordt/V  # past of worden ('become)
    )
    .*      # followed by anything
    van/Prep # followed by van ('of')
"""
VAN = re.compile(vnv, re.VERBOSE)
for doc in conll2002.chunked_sents('ned.train'):
    for r in nltk.sem.extract_rels('PER', 'ORG', doc, corpus='conll2002', pattern=VAN):
        print(nltk.sem.rtuple(r))
        print(nltk.sem.clause(r, relsym="VAN"))

[PER: "Cornet/V d'Elzius/N"] 'is/V op/Prep dit/Pron ogenblik/N kabinetsadviseur/N van/Prep staatssecretaris/N voor/Prep' [ORG: 'Buitenlandse/N Handel/N']
VAN("cornet_d'elzius", 'buitenlandse_handel')
[PER: 'Johan/N Rottiers/N'] 'is/V informaticacoördinator/N van/Prep het/Art' [ORG: 'Kardinaal/N Van/N Roey/N Instituut/N']
VAN('johan_rottiers', 'kardinaal_van_roey_instituut')
[PER: 'Annie/N Lennox/N'] 'wordt/V het/Art concert/N van/Prep' [ORG: 'Eurythmics/N']
VAN('annie_lennox', 'eurythmics')
